In [46]:
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit
import numpy as np

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.74",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

molecule = driver.run()
mapper = ParityMapper()
qubit_op = mapper.map(molecule.hamiltonian.second_q_op())
print("化简前为",qubit_op)
cost_h=qubit_op
cost_h = cost_h.simplify()
print("化简后为",cost_h)


from qiskit.quantum_info import SparsePauliOp

driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0),
    ("IXII", -1.0),
    ("IIXI", -1.0),
    ("IIIX", -1.0),
])
print(driver_h)

from qiskit.quantum_info import SparsePauliOp

def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)
print(comm_h)

from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit import QuantumCircuit
from numpy import pi
from collections import Counter
import matplotlib.pyplot as plt

# ----------- Pauli Gate 演化 -------------
def apply_pauli_evolution(qc, pauli_str, angle):
    n = qc.num_qubits
    pauli_str = pauli_str[::-1]  # Qiskit 是 little-endian
    indices = [i for i, p in enumerate(pauli_str) if p != 'I']
    if not indices:
        return

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.sdg(i)
            qc.h(i)

    if len(indices) > 1:
        for i in range(len(indices) - 1):
            qc.cx(indices[i], indices[i + 1])
        target = indices[-1]
    else:
        target = indices[0]

    qc.rz(2 * angle, target)

    if len(indices) > 1:
        for i in reversed(range(len(indices) - 1)):
            qc.cx(indices[i], indices[i + 1])

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.h(i)
            qc.s(i)


# ----------- 单层 FALQON 结构 -------------
def apply_falqon_layer(qc, beta_k, cost_h, driver_h, delta_t):
    for label, coeff in zip(cost_h.paulis.to_labels(), cost_h.coeffs):
        apply_pauli_evolution(qc, label, float(coeff.real * delta_t))
    for label, coeff in zip(driver_h.paulis.to_labels(), driver_h.coeffs):
        apply_pauli_evolution(qc, label, float(coeff.real * beta_k * delta_t))

from qiskit import transpile


def falqon_qiskit_fast_final_gate_count(n_qubits, n_layers, beta_1, delta_t, cost_h, driver_h, comm_h, 
                                        target_energy=-1.13728383, energy_tol=0.001):
    beta = [beta_1]
    energies = []

    state = Statevector.from_label("+" * n_qubits)
    final_circuit = QuantumCircuit(n_qubits)
    final_circuit.h(range(n_qubits))  # 初始 H^{⊗n}

    for i in range(n_layers):
        qc_layer = QuantumCircuit(n_qubits)
        apply_falqon_layer(qc_layer, beta[i], cost_h, driver_h, delta_t)

        final_circuit.compose(qc_layer, inplace=True)
        state = state.evolve(qc_layer)

        energy = state.expectation_value(cost_h).real
        next_beta = -delta_t * state.expectation_value(comm_h).real

        energies.append(energy)
        beta.append(next_beta)

        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")


        # ✅ 判断是否达到误差容限
        if abs(energy - target_energy) < energy_tol:
            print(f"✅ 满足误差容限 |E - E_FCI| < {energy_tol}, 在第{i+1}步停止演化。")
            break

    return beta, energies, final_circuit


from qiskit import transpile

# 基础门集合（例如 IBM 超导平台）

# 执行主算法
basis = ["ecr", "id", "rz", "sx", "x"]
n_layers = 100
beta_1 = 0.03
delta_t = 0.03
res_beta, res_energies, final_circuit = falqon_qiskit_fast_final_gate_count(
    n_qubits=4,
    n_layers=n_layers,
    beta_1=beta_1,
    delta_t=delta_t,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h
)
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_full(4)
from qiskit.transpiler import CouplingMap

# 定义线性拓扑：0–1–2–3
linear_coupling = CouplingMap(couplinglist=[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2)])
transpiled_circuit= transpile(
    final_circuit,
    basis_gates=basis,  # 典型 IBM 超导平台的基础门集合
    coupling_map=coupling_map,
    optimization_level=1
)


# ✅ 最后统一 transpile 并统计基础门数量
#transpiled_circuit = transpile(final_circuit, basis_gates=basis, optimization_level=1)
final_gate_counts = transpiled_circuit.count_ops()

# ✅ 打印统计
print("\n📊 最终电路在指定基础门集下的门数量统计：")
for gate, count in final_gate_counts.items():
    print(f"{gate}: {count}")





化简前为 SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZZII', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.12062523+0.j,
  0.17141283+0.j,  0.16868898+0.j, -0.22343154+0.j,  0.16592785+0.j,
  0.04530262+0.j, -0.04530262+0.j, -0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])
化简后为 SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZZII', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.12062523+0.j,
  0.17141283+0.j,  0.16868898+0.j, -0.22343154+0.j,  0.16592785+0.j,
  0.04530262+0.j, -0.04530262+0.j, -0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])
SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j])
SparsePauliOp(['YZII', 'YZIZ', 'YXIX', 'YXZX', 'Y

In [29]:
from collections import defaultdict

def count_gates_per_qubit(circuit: QuantumCircuit):
    qubit_gate_counts = defaultdict(lambda: defaultdict(int))  # qubit -> gate -> count

    for instr in circuit.data:
        gate = instr.operation.name
        qubits = instr.qubits
        for q in qubits:
            q_index = circuit.find_bit(q).index  # ✅ 获取 qubit 在电路中的编号
            qubit_gate_counts[q_index][gate] += 1

    return qubit_gate_counts


In [30]:
qubit_gate_stats = count_gates_per_qubit(transpiled_circuit)

print("📊 每个 qubit 的门执行次数统计：")
for qubit, gate_counts in qubit_gate_stats.items():
    print(f"Qubit {qubit}:")
    for gate, count in gate_counts.items():
        print(f"  {gate}: {count}")


📊 每个 qubit 的门执行次数统计：
Qubit 0:
  rz: 33739
  sx: 23245
  ecr: 31032
  x: 3893
Qubit 1:
  rz: 24538
  sx: 7729
  ecr: 18102
  x: 11651
Qubit 2:
  rz: 22560
  sx: 14210
  ecr: 23274
  x: 4478
Qubit 3:
  rz: 21348
  sx: 18073
  ecr: 15516
  x: 1994


In [27]:
a=2.0487e-04
b=1-a
c=pow(b,5171)
print(c)

d=2.5529e-04
e=1-d
f=pow(e,19396)
print(f)


0.3466320963003955
0.007067581087344809


In [16]:
from qiskit.providers.backend import Backend
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
import numpy as np

def compute_expected_fidelity(circuit: QuantumCircuit, backend: Backend, optimization_level: int = 1) -> float:
    """
    自动估算电路在指定 backend 上的 expected fidelity（期望保真度）
    :param circuit: Qiskit QuantumCircuit
    :param backend: Qiskit Backend（必须支持 properties()）
    :param optimization_level: 编译优化等级，默认 1
    :return: 估算的 expected fidelity 值（越接近1越好）
    """

    # Transpile circuit for the backend
    transpiled_circuit = transpile(circuit, backend=backend, optimization_level=optimization_level)

    # 获取电路中所有操作（门）
    ops = transpiled_circuit.count_ops()
    basis_gates = backend.configuration().basis_gates
    print(f"基础门集合: {basis_gates}")
    # 获取 backend 上的门错误率（按每种门类型取中位数）
    props = backend.properties()
    gate_errors = {}

    # 支持常见基础门

    for gate in basis_gates:
        errs = []
        for g in props.gates:
            if g.gate == gate:
                errs.append(g.parameters[0].value)
        if errs:
            gate_errors[gate] = np.median(errs)

    # 估算保真度：每个门类型 (1 - ε)^count
    fidelity = 1.0
    for gate, count in ops.items():
        error_rate = gate_errors.get(gate.lower(), 0.0)
        fidelity *= (1 - error_rate) ** count

    return fidelity


In [12]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from IPython.display import display, Latex
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import Sampler
from qiskit import transpile
import numpy as np
from qiskit_ibm_runtime import SamplerV2 as Sampler




In [13]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService (channel="ibm_quantum",
                                token="304636e08832e99bf5cd66f58d0f1180f193cd79e97987083aafe0ba33cef2f9ceb8ac2bba91aa24b612787451d32e9dc7206edf2e4347ba88443ba5261f04a6")

QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token="304636e08832e99bf5cd66f58d0f1180f193cd79e97987083aafe0ba33cef2f9ceb8ac2bba91aa24b612787451d32e9dc7206edf2e4347ba88443ba5261f04a6",
                                  overwrite=True)

In [14]:
backends = service.backends()
for backend in backends:
    print(backend.name)

ibm_brisbane
ibm_sherbrooke


In [17]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")  # 替换为任意 backend 名称

expected_fid = compute_expected_fidelity(final_circuit, backend)
print(f"✅ 估算保真度：{expected_fid:.6f}")


基础门集合: ['ecr', 'id', 'rz', 'sx', 'x']
✅ 估算保真度：0.000000


In [18]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")  # 替换为任意 backend 名称
props = backend.properties()

# 打印所有门的误差信息
for g in props.gates:
    gate_type = g.gate
    qubits = g.qubits
    error_rate = g.parameters[0].value
    print(f"{gate_type} on qubits {qubits} has error {error_rate:.4e}")


id on qubits [0] has error 2.0487e-04
id on qubits [1] has error 2.8156e-03
id on qubits [2] has error 1.5231e-04
id on qubits [3] has error 2.9485e-04
id on qubits [4] has error 1.5232e-04
id on qubits [5] has error 3.0419e-04
id on qubits [6] has error 1.2795e-04
id on qubits [7] has error 1.0023e-03
id on qubits [8] has error 1.2264e-04
id on qubits [9] has error 3.9404e-04
id on qubits [10] has error 2.0830e-04
id on qubits [11] has error 1.5545e-04
id on qubits [12] has error 2.8481e-04
id on qubits [13] has error 1.9751e-04
id on qubits [14] has error 2.3262e-04
id on qubits [15] has error 2.7927e-04
id on qubits [16] has error 5.2343e-04
id on qubits [17] has error 1.1958e-04
id on qubits [18] has error 2.3013e-04
id on qubits [19] has error 1.7112e-04
id on qubits [20] has error 1.1153e-04
id on qubits [21] has error 1.6002e-04
id on qubits [22] has error 2.5529e-04
id on qubits [23] has error 1.8559e-04
id on qubits [24] has error 2.3076e-04
id on qubits [25] has error 3.4503e

In [47]:
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit
import numpy as np

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.74",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

molecule = driver.run()
mapper = ParityMapper()
qubit_op = mapper.map(molecule.hamiltonian.second_q_op())
print("化简前为",qubit_op)
cost_h=qubit_op
cost_h = cost_h.simplify()
print("化简后为",cost_h)


from qiskit.quantum_info import SparsePauliOp

driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0),
    ("IXII", -1.0),
    ("IIXI", -1.0),
    ("IIIX", -1.0),
])
print(driver_h)

from qiskit.quantum_info import SparsePauliOp

def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)
print(comm_h)

from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit import QuantumCircuit
from numpy import pi
from collections import Counter
import matplotlib.pyplot as plt

# ----------- Pauli Gate 演化 -------------
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import Pauli

from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import Pauli

def apply_pauli_evolution_gate(qc, pauli_str, angle):
    pauli_op = Pauli(pauli_str)
    qubits = list(range(len(pauli_str)))  # ✅ 提供所有 qubit index
    gate = PauliEvolutionGate(pauli_op, time=angle)
    qc.append(gate, qubits)


# ----------- 单层 FALQON 结构 -------------
def apply_falqon_layer(qc, beta_k, cost_h, driver_h, delta_t, coeff_threshold=0.0):
    for label, coeff in zip(cost_h.paulis.to_labels(), cost_h.coeffs):
        if abs(coeff.real) < coeff_threshold:
            continue
        apply_pauli_evolution_gate(qc, label, float(coeff.real * delta_t))
    for label, coeff in zip(driver_h.paulis.to_labels(), driver_h.coeffs):
        apply_pauli_evolution_gate(qc, label, float(coeff.real * beta_k * delta_t))


from qiskit import transpile


def falqon_qiskit_fast_final_gate_count(n_qubits, n_layers, beta_1, delta_t, cost_h, driver_h, comm_h, 
                                        target_energy=-1.13728383, energy_tol=0.001):
    beta = [beta_1]
    energies = []

    state = Statevector.from_label("+" * n_qubits)
    final_circuit = QuantumCircuit(n_qubits)
    final_circuit.h(range(n_qubits))  # 初始 H^{⊗n}

    for i in range(n_layers):
        qc_layer = QuantumCircuit(n_qubits)
        apply_falqon_layer(qc_layer, beta[i], cost_h, driver_h, delta_t)

        final_circuit.compose(qc_layer, inplace=True)
        state = state.evolve(qc_layer)

        energy = state.expectation_value(cost_h).real
        next_beta = -delta_t * state.expectation_value(comm_h).real

        energies.append(energy)
        beta.append(next_beta)

        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")


        # ✅ 判断是否达到误差容限
        if abs(energy - target_energy) < energy_tol:
            print(f"✅ 满足误差容限 |E - E_FCI| < {energy_tol}, 在第{i+1}步停止演化。")
            break

    return beta, energies, final_circuit


from qiskit import transpile

# 基础门集合（例如 IBM 超导平台）

# 执行主算法
basis = ["ecr", "id", "rz", "sx", "x"]
n_layers = 100
beta_1 = 0.03
delta_t = 0.03
res_beta, res_energies, final_circuit = falqon_qiskit_fast_final_gate_count(
    n_qubits=4,
    n_layers=n_layers,
    beta_1=beta_1,
    delta_t=delta_t,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h
)
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_full(4)
from qiskit.transpiler import CouplingMap

# 定义线性拓扑：0–1–2–3
linear_coupling = CouplingMap(couplinglist=[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2)])
transpiled_circuit= transpile(
    final_circuit,
    basis_gates=basis,  # 典型 IBM 超导平台的基础门集合
    coupling_map=coupling_map,
    optimization_level=1
)


# ✅ 最后统一 transpile 并统计基础门数量
#transpiled_circuit = transpile(final_circuit, basis_gates=basis, optimization_level=1)
final_gate_counts = transpiled_circuit.count_ops()

# ✅ 打印统计
print("\n📊 最终电路在指定基础门集下的门数量统计：")
for gate, count in final_gate_counts.items():
    print(f"{gate}: {count}")




化简前为 SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZZII', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.12062523+0.j,
  0.17141283+0.j,  0.16868898+0.j, -0.22343154+0.j,  0.16592785+0.j,
  0.04530262+0.j, -0.04530262+0.j, -0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])
化简后为 SparsePauliOp(['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZZII', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI'],
              coeffs=[-0.81217061+0.j,  0.17141283+0.j, -0.22343154+0.j,  0.12062523+0.j,
  0.17141283+0.j,  0.16868898+0.j, -0.22343154+0.j,  0.16592785+0.j,
  0.04530262+0.j, -0.04530262+0.j, -0.04530262+0.j,  0.04530262+0.j,
  0.16592785+0.j,  0.17441288+0.j,  0.12062523+0.j])
SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-1.+0.j, -1.+0.j, -1.+0.j, -1.+0.j])
SparsePauliOp(['YZII', 'YZIZ', 'YXIX', 'YXZX', 'Y